In [ ]:
!pip install numpyro

In [ ]:
!pip install daft --upgrade

In [ ]:
#@title chilis example
import matplotlib.pyplot as plt
import pandas as pd
from functools import partial, partialmethod
import daft   ### %pip install -U git+https://github.com/daft-dev/daft.git
from numpy.random import default_rng
import numpy as np

class dag(daft.PGM):
    def __init__(self, *args, **kwargs):
        daft.PGM.__init__(self, *args, **kwargs)

    obsNode = partialmethod(daft.PGM.add_node, scale = 1.3, aspect = 2.4, fontsize = 10, plot_params = {'facecolor': 'cadetblue'})
    decNode = partialmethod(daft.PGM.add_node, aspect = 2.2, fontsize = 10, shape = "rectangle", plot_params = {'facecolor': 'thistle'})
    detNode = partialmethod(daft.PGM.add_node, aspect = 5.4, fontsize = 9.25, alternate = True, plot_params = {'facecolor': 'aliceblue'})
    latNode = partialmethod(daft.PGM.add_node, scale = 1.3, aspect = 2.4, fontsize = 10, plot_params = {'facecolor': 'aliceblue'})
    detNodeBig = partialmethod(daft.PGM.add_node, scale = 1.6, aspect = 2.25, fontsize = 10, alternate = True, plot_params = {'facecolor': 'aliceblue'})
    latNodeBig = partialmethod(daft.PGM.add_node, scale = 1.6, aspect = 2.2, fontsize = 10, plot_params = {'facecolor': 'aliceblue'})

pgm = dag(dpi = 300, alternate_style="outer")
pgm.obsNode("x","Sales \n"+r"Increase $(x)$",1,1)
pgm.latNode("theta","Success\n"+r"Probability $(\theta)$",1,2)
pgm.add_edge("theta","x")
pgm.add_plate([0.1, 0.2, 1.8, 1.25], label = "Observation:\n" + r"$i = 1, 2, \ldots, N$",
              label_offset = (2,2), rect_params = dict({"fill": False, "linestyle": "dashed", "edgecolor": "black"}))
pgm.show(dpi = 120)

In [ ]:
import numpy as np
import numpyro
import numpyro.distributions as dist

## define the graphical/statistical model as a Python function
## pass data and the cardinality of plates as inputs
## N represents the number of chilis stores for which data is observed
def chilisModel(x):
    # numpyro.sample is a "primitive", i.e. basic building block of model
    theta = numpyro.sample('theta', dist.Uniform(low = 0, high = 1))
    # numpyro.plate is another primitive
    with numpyro.plate('observation', len(x)):
        x = numpyro.sample('x', dist.Bernoulli(probs = theta), obs=x)


In [ ]:
## define the required inputs for chilismodel
## assume first two stores are a success and
## the third store is not (i.e. make up some data)
salesIncData = np.array([1,1,0])

In [ ]:
from jax import random
from numpyro.infer import MCMC, NUTS

## computationally get posterior distribution
## in the below line, only change "chilisModel" for different problems
mcmc = MCMC(NUTS(chilisModel), num_warmup=500, num_samples=4000)
rng_key = random.PRNGKey(seed = 111) ## so you and I get same results
## supply the run method with a random key followed by arguments to model
mcmc.run(rng_key, x=salesIncData) ## get representative sample of posterior

In [ ]:
## get summary statistics
mcmc.print_summary()

In [ ]:
## get samples into xarray
drawsDS = az.from_numpyro(mcmc).posterior

In [ ]:
drawsDS

In [ ]:
## use indicator function to make probablistic statements
## for example, find P(theta > 0.5)

(
    drawsDS
    .assign(thetaOver50 = drawsDS.theta > 0.50)
    .mean() # about 60% probability theta is over 50%
).to_pandas()


In [ ]:
# arviz: see https://python.arviz.org/en/stable/
import arviz as az

In [ ]:
## graph density function from representative sample
#| label: fig-repBeta2
#| echo: false
#| fig-cap: A probability density estimate (purple line) for a beta(6,2) distribution.
#| fig-align: left
#| results: hide
#| collapse: true

from numpy import linspace

fig, ax = plt.subplots(figsize=(6, 3.5),
                        layout='constrained')

# plot histogram
az.plot_dist(drawsDS.theta, kind = "hist", color = "cadetblue", ax = ax,
             hist_kwargs = {"bins": linspace(0,1,30), "density": True})

# plot density estimate, i.e. estimate of f(x)
az.plot_dist(drawsDS.theta, ax = ax, color = "darkorchid",
             plot_kwargs = {"zorder": 1, "linewidth": 4})
ax.set_xticks([0,.25,.5,.75,1])

ax.set_ylabel('Plausibility Measure: ' + r'$f_\Theta(\theta)$')
ax.set_xlabel(r'$\theta$')

plt.show()

In [ ]:
## quick and easy way to plot
az.plot_dist(drawsDS.theta)

## YOUR TURN



In [ ]:
#@title modifying the chilis example
import matplotlib.pyplot as plt
import pandas as pd
from functools import partial, partialmethod
import daft   ### %pip install -U git+https://github.com/daft-dev/daft.git
from numpy.random import default_rng
import numpy as np

class dag(daft.PGM):
    def __init__(self, *args, **kwargs):
        daft.PGM.__init__(self, *args, **kwargs)

    obsNode = partialmethod(daft.PGM.add_node, scale = 1.3, aspect = 2.4, fontsize = 10, plot_params = {'facecolor': 'cadetblue'})
    decNode = partialmethod(daft.PGM.add_node, aspect = 2.2, fontsize = 10, shape = "rectangle", plot_params = {'facecolor': 'thistle'})
    detNode = partialmethod(daft.PGM.add_node, aspect = 5.4, fontsize = 9.25, alternate = True, plot_params = {'facecolor': 'aliceblue'})
    latNode = partialmethod(daft.PGM.add_node, scale = 1.3, aspect = 2.4, fontsize = 10, plot_params = {'facecolor': 'aliceblue'})
    detNodeBig = partialmethod(daft.PGM.add_node, scale = 1.6, aspect = 2.25, fontsize = 10, alternate = True, plot_params = {'facecolor': 'aliceblue'})
    latNodeBig = partialmethod(daft.PGM.add_node, scale = 1.6, aspect = 2.2, fontsize = 10, plot_params = {'facecolor': 'aliceblue'})

pgm = dag(dpi = 300, alternate_style="outer")
pgm.obsNode("x","Sales \n"+r"Increase $(x)$",1,1)
pgm.latNode("theta","Success\n"+r"Probability $(\theta)$",1,2)
pgm.add_edge("theta","x")
pgm.add_plate([0.1, 0.2, 1.8, 1.25], label = "Observation:\n" + r"$i = 1, 2, \ldots, N$",
              label_offset = (2,2), rect_params = dict({"fill": False, "linestyle": "dashed", "edgecolor": "black"}))
pgm.show(dpi = 120)

Use the following statistical model:

$$
\begin{aligned}
\Theta \equiv& \textrm{ Store Success Probability: } \\
X \equiv& \textrm{ Sales Increase: } \\
        & \textrm{ If sales increase more than 5} \% \textrm{, then }X=1 \textrm{ otherwise, }X=0.\\
\theta \sim  & \textrm{ Beta}(2,1) \\
x \sim  & \textrm{ Bernoulli}(\theta)
\end{aligned}
$$

Q1: Does this prior model suggest success or failure to be more likely?

Q2: Write code and answer what is the posterior probability that $\theta \leq 40%$?